#How To Build a Model: Step By Step


In [1]:
#Pytorch Imports
%load_ext autoreload
%autoreload 2
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data import get_tokenizer
from torchtext.datasets import IMDB
from torchtext.transforms import ToTensor
from torchtext.vocab import build_vocab_from_iterator
#Pandas
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
# from sklearn.model_selection import train_test_split
import time




In [2]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from transformers import AutoTokenizer

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

Pick Your Universal Hyperparamaters:

In [3]:
batch_size = 64
epochs = 100
input_dim = 512
hidden_dim = 100
output_dim = 10
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

# Setup Your Model Workspace:

Your model architecture, and all of it's relevent code, will go in a folder at the location ./NNDL/(Your Model Name Here)


# Setup Your Dataset/Data Loaders Here

Pytorch usually takes a csv file. You need to write a function( For naming convention let's call it *create_torch_datasets()* ) that takes the universal dataset kyle got from parsing and seperates/loads it into data that you want your model to train on




In [4]:
#Create Custom Dataset Class:
class TextDataset(Dataset):
    def __init__(self, csv_file, root_dir):
        self.root_dir = root_dir
        self.data = pd.read_csv(csv_file)
        self.mapping = {}
        self.cur_map = 1
        self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
    def __len__(self):
        return len(self.data)
    def __getitem__(self, index):
        text_o = os.path.join(self.root_dir,
            self.data.iloc[index, 0])
        tokens = self.tokenizer(text_o, padding='max_length',truncation=True)['input_ids']
        # print(f"==>> tokens: {tokens}")
        # token_ids = self.tokenizer.convert_tokens_to_ids(tokens)
        # print(f"==>> token_ids: {token_ids}")
        
        #input_tensor = torch.tensor(self.tokenizer.convert_tokens_to_ids(tokens)).unsqueeze(0)
        category_text = os.path.join(self.root_dir,
            self.data.iloc[index, 1])
        if category_text in self.mapping:
            category = self.mapping[category_text]
        else:
            self.mapping[category_text] = self.cur_map
            category = self.cur_map
            self.cur_map+=1
        # print(f"self.mapping:{self.mapping}")
        return torch.tensor(tokens,dtype=torch.float32),category

In [5]:
from transformers import BertTokenizer

# Load the pre-trained BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Input text
text = "Hello, how are you doing?"

# Tokenize the text
tokens = tokenizer.tokenize(text)

# Convert tokens to numerical IDs
token_ids = tokenizer.convert_tokens_to_ids(tokens)

# Print the results
print("Original Text:", text)
print("Tokens:", tokens)
print("Token IDs:", torch.tensor(token_ids))

Original Text: Hello, how are you doing?
Tokens: ['hello', ',', 'how', 'are', 'you', 'doing', '?']
Token IDs: tensor([7592, 1010, 2129, 2024, 2017, 2725, 1029])


In [6]:
#This is an example of what I'm talking about. The Fashion dataset is pretty easy bc it's function alr exists
# but you need to create your own function to make the training data and test_ data datasets
text_dataset = TextDataset(csv_file=r'data\products.csv [MConverter.eu].csv',root_dir='/')
for i, sample in enumerate(text_dataset):
    print(i, len(sample[0]), sample[1])
generator1 = torch.Generator().manual_seed(42)
train,test = torch.utils.data.random_split(text_dataset,[0.8,0.2], generator=generator1)
train_dataloader =  DataLoader(train, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)
test_dataloader =  DataLoader(test, batch_size=batch_size,
                        shuffle=True, num_workers=0, drop_last=True)

# train_iter= IMDB(split="train")
# test_iter = IMDB(split="test")
# tokenizer = get_tokenizer("basic_english")
# def yield_tokens(data_iter):
#     for _, text in data_iter:
#         yield tokenizer(text)
# train_tokens = []
# vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
# vocab.set_default_index(vocab["<unk>"])
# test_tokens = []
# test_tokens = build_vocab_from_iterator(yield_tokens(test_iter), specials=["<unk>"])



0 512 1
1 512 2
2 512 2
3 512 2
4 512 3
5 512 4
6 512 2
7 512 4
8 512 4
9 512 4
10 512 4
11 512 4
12 512 4
13 512 4
14 512 4
15 512 4
16 512 2
17 512 4
18 512 4
19 512 4
20 512 4
21 512 4
22 512 4
23 512 4
24 512 4
25 512 4
26 512 4
27 512 4
28 512 4
29 512 4
30 512 4
31 512 3
32 512 3
33 512 4
34 512 4
35 512 4
36 512 4
37 512 4
38 512 4
39 512 4
40 512 1
41 512 3
42 512 4
43 512 4
44 512 4
45 512 4
46 512 5
47 512 4
48 512 4
49 512 2
50 512 4
51 512 4
52 512 4
53 512 4
54 512 4
55 512 4
56 512 4
57 512 4
58 512 4
59 512 4
60 512 3
61 512 3
62 512 4
63 512 4
64 512 4
65 512 4
66 512 4
67 512 4
68 512 3
69 512 4
70 512 4
71 512 4
72 512 4
73 512 4
74 512 4
75 512 4
76 512 4
77 512 4
78 512 4
79 512 4
80 512 4
81 512 4
82 512 4
83 512 4
84 512 4
85 512 4
86 512 4
87 512 4
88 512 6
89 512 4
90 512 4
91 512 4
92 512 4
93 512 3
94 512 4
95 512 4
96 512 4
97 512 4
98 512 4
99 512 4
100 512 4
101 512 4
102 512 4
103 512 4
104 512 4
105 512 6
106 512 4
107 512 4
108 512 4
109 512 4
110 512 4


Put the dataset into data loader, and check the shape, make sure it's how you want it.


In [7]:
# Create data loaders.
#train_dataloader = DataLoader(text_dataset, batch_size=batch_size,shuffle=True, num_workers=0)
#print(train_dataloader[0])
# for i_batch, sample_batched in enumerate(train_dataloader):
#   print(f"==>> i_batch: {i_batch}")
#   print(f"==>> sample_batched: {len(sample_batched)}")
      #print(i_batch, sample_batched['text'][0][0].item(),
        #sample_batched['category'])

# Display image and label.
train_features, train_labels = next(iter(train_dataloader))
print(train_features)
print(train_labels)
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
# img = train_features[0].squeeze()
# label = train_labels[0]
# plt.imshow(img, cmap="gray")
# plt.show()
# print(f"Label: {label}")

tensor([[  101.,   120.,  1884.,  ...,     0.,     0.,     0.],
        [  101.,   120.,  5444.,  ...,     0.,     0.,     0.],
        [  101.,   120.,  2555.,  ...,     0.,     0.,     0.],
        ...,
        [  101.,   120., 25590.,  ...,     0.,     0.,     0.],
        [  101.,   120.,  2298.,  ...,     0.,     0.,     0.],
        [  101.,   120., 14044.,  ...,     0.,     0.,     0.]])
tensor([4, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        3, 4, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4])
Feature batch shape: torch.Size([64, 512])
Labels batch shape: torch.Size([64])


# Setup Model Architecture

1. Create your model architecture in your folder
2. Pick your loss function and optimizer

In [8]:
from NNDL.RNN.architecture import RNN
from NNDL.Utils.solver import train,test


model = RNN(input_dim, hidden_dim, output_dim).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)


# Train the model
This should work to train your model. We may have to make some edits for different optimizations, but we can figure it out.


In [9]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.530937  [   64/  400]
loss: 2.458644  [  128/  400]
loss: 2.491678  [  192/  400]
loss: 2.486065  [  256/  400]
loss: 2.428947  [  320/  400]
loss: 2.468772  [  384/  400]
Test Error: 
 Accuracy: 1.0%, Avg loss: 2.453056 

Epoch 2
-------------------------------
loss: 2.446746  [   64/  400]
loss: 2.409442  [  128/  400]
loss: 2.423259  [  192/  400]
loss: 2.430985  [  256/  400]
loss: 2.429370  [  320/  400]
loss: 2.391178  [  384/  400]
Test Error: 
 Accuracy: 1.0%, Avg loss: 2.398261 

Epoch 3
-------------------------------
loss: 2.375946  [   64/  400]
loss: 2.371090  [  128/  400]
loss: 2.364766  [  192/  400]
loss: 2.367434  [  256/  400]
loss: 2.396024  [  320/  400]
loss: 2.355512  [  384/  400]
Test Error: 
 Accuracy: 1.0%, Avg loss: 2.362857 

Epoch 4
-------------------------------
loss: 2.358243  [   64/  400]
loss: 2.314357  [  128/  400]
loss: 2.329631  [  192/  400]
loss: 2.312750  [  256/  400]
loss: 2.324426  [  320/  40